In [0]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('nlp_tools').getOrCreate()
from pyspark.ml.feature import Tokenizer,RegexTokenizer
from pyspark.sql.functions import col,udf
from pyspark.sql.types import IntegerType

In [0]:
#We will create a data-frame full of sentences
sen_def = spark.createDataFrame([
    

In [0]:
sen_def.show()

+---+--------------------+
| id|            sentence|
+---+--------------------+
|  0|Hi I heard about ...|
|  1|I wish Java could...|
|  2|Logistic,regressi...|
+---+--------------------+



In [0]:
#We will create our Tokenizer objects
tokenizer = Tokenizer(inputCol='sentence',outputCol='words')

In [0]:
#Regular expression tokenizer
regex_tokenizer = RegexTokenizer(inputCol='sentence',outputCol='words',pattern='\\W')

In [0]:
#user define function
count_tokens = udf(lambda words:len(words),IntegerType())

In [0]:
tokenized = tokenizer.transform(sen_def)

In [0]:
tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic,regress...|     1|
+---+--------------------+--------------------+------+



In [0]:
rg_tokenized = regex_tokenizer.transform(sen_def)

In [0]:
rg_tokenized.withColumn('tokens',count_tokens(col('words'))).show()

+---+--------------------+--------------------+------+
| id|            sentence|               words|tokens|
+---+--------------------+--------------------+------+
|  0|Hi I heard about ...|[hi, i, heard, ab...|     5|
|  1|I wish Java could...|[i, wish, java, c...|     7|
|  2|Logistic,regressi...|[logistic, regres...|     5|
+---+--------------------+--------------------+------+



In [0]:
#some words dont need to be considers , we can use a function to not count them
#We filter common words like: I,the,had,a,etc.   Or add additional stop words.
from pyspark.ml.feature import StopWordsRemover

In [0]:
sentenceDataFrame = spark.createDataFrame([
    (0,['I','saw','the','green','horse']),
    (1,['Mary','had','a','little','lamb'])
],['id','tokens'])

In [0]:
remover = StopWordsRemover(inputCol='tokens',outputCol='filtered')

In [0]:
remover.transform(sentenceDataFrame).show()

+---+--------------------+--------------------+
| id|              tokens|            filtered|
+---+--------------------+--------------------+
|  0|[I, saw, the, gre...| [saw, green, horse]|
|  1|[Mary, had, a, li...|[Mary, little, lamb]|
+---+--------------------+--------------------+



In [0]:
#n-grams  
from pyspark.ml.feature import NGram
word_dataframe = spark.createDataFrame([
    (0,['Hi','I','heard','about','Spark']),
    (1,['I','wish','Java','could','use','case','classes']),
    (2,['Logistic','regression','models','are','neat'])
],['id','words'])

In [0]:
ngram = NGram(n=3,inputCol='words',outputCol='grams')

In [0]:
ngram.transform(word_dataframe).select('grams').show(truncate=False)

+--------------------------------------------------------------------------------+
|grams                                                                           |
+--------------------------------------------------------------------------------+
|[Hi I heard, I heard about, heard about Spark]                                  |
|[I wish Java, wish Java could, Java could use, could use case, use case classes]|
|[Logistic regression models, regression models are, models are neat]            |
+--------------------------------------------------------------------------------+



In [0]:
#############################  PART 2  ###########################################

In [0]:
from pyspark.ml.feature import HashingTF,IDF,Tokenizer

In [0]:
sentenceData = spark.createDataFrame([
    (0.0,'Hi I heard about Spark'),
    (0.0,'I wish Java could use case classes'),
    (1.0,'Logistic regression models are neat')
],['label','sentence'])

In [0]:
sentenceData.show()

+-----+--------------------+
|label|            sentence|
+-----+--------------------+
|  0.0|Hi I heard about ...|
|  0.0|I wish Java could...|
|  1.0|Logistic regressi...|
+-----+--------------------+



In [0]:
#now we will tokenize it
tokenizer1 = Tokenizer(inputCol='sentence',outputCol='words')

In [0]:
words_data = tokenizer1.transform(sentenceData)

In [0]:
words_data.show(truncate=False)

+-----+-----------------------------------+------------------------------------------+
|label|sentence                           |words                                     |
+-----+-----------------------------------+------------------------------------------+
|0.0  |Hi I heard about Spark             |[hi, i, heard, about, spark]              |
|0.0  |I wish Java could use case classes |[i, wish, java, could, use, case, classes]|
|1.0  |Logistic regression models are neat|[logistic, regression, models, are, neat] |
+-----+-----------------------------------+------------------------------------------+



In [0]:
hashing_tf = HashingTF(inputCol='words',outputCol='rawFeatures')

In [0]:
featurised_data = hashing_tf.transform(word_data)

In [0]:
idf = IDF(inputCol='rawFeatures',outputCol='features')

In [0]:
idf_model = idf.fit(featurised_data)

In [0]:
rescaled_data = idf_model.transform(featurised_data)

In [0]:
rescaled_data.select('label','features').show(truncate=False)

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|features                                                                                                                                                                                      |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0.0  |(262144,[18700,19036,33808,66273,173558],[0.6931471805599453,0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453])                                                   |
|0.0  |(262144,[19036,20719,55551,58672,98717,109547,192310],[0.28768207245178085,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453,0.6931471805599453])|
|1.0 

In [0]:
# Now it is in form of 'label' and 'features' and we can deal with it with any supervised learning algorithm

In [0]:
from pyspark.ml.feature import CountVectorizer

In [0]:
df = spark.createDataFrame([
    (0,'a b c'.split(' ')),
    (1,'a b b c a'.split(' '))
],['id','words count'])

In [0]:
df.show()

+---+---------------+
| id|    words count|
+---+---------------+
|  0|      [a, b, c]|
|  1|[a, b, b, c, a]|
+---+---------------+



In [0]:
cv = CountVectorizer(inputCol='words count',outputCol='features',vocabSize=3,minDF=2.0)

In [0]:
model = cv.fit(df)

In [0]:
result = model.transform(df)

In [0]:
result.show(truncate=False)

+---+---------------+-------------------------+
|id |words count    |features                 |
+---+---------------+-------------------------+
|0  |[a, b, c]      |(3,[0,1,2],[1.0,1.0,1.0])|
|1  |[a, b, b, c, a]|(3,[0,1,2],[2.0,2.0,1.0])|
+---+---------------+-------------------------+

